In [ ]:
import tweepy
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
from langchain.llms import OpenAI

# Initialize LangChain with OpenAI (assuming you have access)
llm = OpenAI(api_key='YOUR_OPENAI_API_KEY')

# Load a pre-trained sentiment analysis model
nlp = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# X API 2 credentials
bearer_token = 'YOUR_BEARER_TOKEN'

# Authenticate with X API 2
client = tweepy.Client(bearer_token)

# Fetch tweets
query = "YourSearchTerm"
tweets = client.search_recent_tweets(query=query, max_results=100, tweet_fields=["lang"], expansions=["author_id"], user_fields=["username"])

# Extract tweet texts
tweets_data = [tweet.text for tweet in tweets.data if tweet.lang == "en"]

# Convert tweets to DataFrame
df_tweets = pd.DataFrame(tweets_data, columns=['Tweet'])

# Define a function to preprocess tweet text
def preprocess_tweet(tweet):
    import re
    # Remove URLs, mentions, hashtags and convert to lowercase
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r"@\S+|#\S+", '', tweet, flags=re.MULTILINE)
    tweet = tweet.lower()
    return tweet

# Apply preprocessing to each tweet
df_tweets['Processed_Tweet'] = df_tweets['Tweet'].apply(preprocess_tweet)

# Define a function to analyze sentiment using OpenAI
def analyze_tweet(tweet):
    response = llm.invoke(f"What is the sentiment of this tweet? '{tweet}'")
    return response

# Apply semantic analysis to each processed tweet
df_tweets['Analysis'] = df_tweets['Processed_Tweet'].apply(analyze_tweet)

# Count sentiment types
sentiment_counts = df_tweets['Analysis'].value_counts()

# Visualization using matplotlib
sentiment_counts.plot(kind='bar')
plt.title('Tweet Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()
